### **Preparing**

In [ ]:
# install library

!pip install ultralytics --quiet
!pip install roboflow --quiet

In [ ]:
# cek direktori saat ini

import os
HOME = os.getcwd()
print(HOME)

In [ ]:
# mengecek status Ultralytics pada perangkat

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="KoKZnMIgPLGBnC1umdit")
project = rf.workspace("usibi-image-translate").project("usibi-jueew")
version = project.version(2)
dataset = version.download("yolov8")

### **Training**

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data={dataset.location}/data.yaml epochs=20 imgsz=640 plots=True optimizer=Adam

### **Saved Model**


In [ ]:
!yolo export model={HOME}/runs/detect/train/weights/best.pt format=onnx

### **Predict**

In [ ]:
!yolo task=detect mode=predict model='/content/runs/detect/train/weights/best.pt' show=True conf=0.5 source='/content/USIBI-1/test/images'

In [ ]:
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train/weights/best.onnx')

def upload_image():
    uploaded = files.upload()
    for file_name in uploaded.keys():
        return file_name

def predict_and_display(img_path):
    img = cv2.imread(img_path)
    results = model(img)

    if results:
        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                class_id = box.cls[0]
                label = f'{model.names[int(class_id)]}: {confidence:.2f}'
                img = cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                img = cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                print(f'Predicted class: {model.names[int(class_id)]} with confidence: {confidence:.2f}')
    cv2_imshow(img)


def main():
    while True:
        print("Upload an image to test the YOLO model.")
        img_path = upload_image()
        if img_path:
            predict_and_display(img_path)
        else:
            print("No image selected!")
        print("\n")

if __name__ == "__main__":
    main()